In [57]:
import os
import openai
import sys
import json
import random
from IPython.display import display, HTML
sys.path.append('../..')

In [58]:
openai.api_key = os.environ['OPENAI_API_KEY']

## LangChain Imports

In [59]:
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableMap

# https://python.langchain.com/docs/modules/memory/adding_memory
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema import SystemMessage
from langchain_openai import ChatOpenAI

## Setup and Initialize Chain (v1.0)

In [122]:
system_v1_template = """You are an intelligent Cognitive UI agent that is having a conversation with an user.
Your task is to have a conversation and respond appropriately to the user. You have the ability to generate UI components 
in the form of HTML code and HTML is the output format that the user prefers when needed. Before you output any HTML, 
you need to confirm that the HTML is valid, stylized with CSS or other frameworks and most importantly, it must satisfy the user's 
requirements. When you output HTML, you must say nothing else and only the HTML code so that the user 
can render it. You must remember to ask if the user is done customizing before outputing HTML and only output HTML when they're ready.

If the user provides feedback to the generated HTML, you will incorporate their feedback
and make new attempts until they're satisfied. In order to generate HTML, you may need content from the user and it is okay to 
collect information from the user based on the type of UI component they need. 
"""

In [123]:
def make_chat(system_template, model_temp = 0, verbose_mode=False):
    prompt = ChatPromptTemplate.from_messages(
        [
            SystemMessage(
                content=system_template
            ),  # The persistent system prompt
            MessagesPlaceholder(
                variable_name="chat_history"
            ),  # Where the memory will be stored.
            HumanMessagePromptTemplate.from_template(
                "{human_input}"
            ),  # Where the human input will injected
        ]
    )
    
    llm = ChatOpenAI(temperature = model_temp)
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    
    chat_llm_chain = LLMChain(
        llm=llm,
        prompt=prompt,
        verbose=verbose_mode,
        memory=memory,
    )
    return chat_llm_chain

### Testing v1.0

In [124]:
chat_llm_chain = make_chat(system_v1_template)
chat_llm_chain.predict(human_input="Hi")

'Hello! How can I assist you today?'

In [125]:
chat_llm_chain.predict(human_input="I want to make a form on my website")

'Great! What fields would you like to include in your form?'

In [126]:
chat_llm_chain.predict(human_input="""I need text fields for their first name, last name, and email address. 
I also need a textbox for the user to input a custom message.""")

'Got it! Would you like any labels for these fields?'

In [127]:
chat_llm_chain.predict(human_input="""
The labels should look fancy and each have different colors that look great together. The label's text should indicate the content 
that the user should put into the text field. The labels should have a Bootstrap Glyphicon representing that particular field if possible.
""")

"Understood! I will create the form with fancy labels, different colors, and Bootstrap Glyphicons for each field. \nI will provide you with the HTML once it's ready. Let me know if you have any specific colors in mind for the labels."

In [128]:
chat_llm_chain.predict(human_input="""I want all the text fields to be on the same row. The textbox on its own row. 
I also want a submit button for the form""")

"Got it! I will arrange the text fields on the same row, place the textbox on a separate row, and include a submit button for the form. \nI will provide you with the HTML once it's ready. Let me know if you have any specific styling preferences for the submit button."

In [129]:
chat_llm_chain.predict(human_input="""The form should have its own background. the form itself should have circular-rounded borders.
The submit button should be horizontally centered and rounded as well. Give the form a modern style!""")

"I will create a modern-styled form with circular-rounded borders, a background color, and a horizontally centered, rounded submit button. \nI will provide you with the HTML once it's ready. Let me know if you have any specific color preferences for the form background and submit button."

In [130]:
chat_llm_chain.predict(human_input="""is the HTML ready?""")

"Not yet. I will provide you with the HTML once it's ready. Thank you for your patience."

In [131]:
chat_llm_chain.predict(human_input="""I would like the HTML now""")

'```html\n<style>\n    .form-container {\n        background-color: #f2f2f2;\n        border-radius: 20px;\n        padding: 20px;\n        width: 50%;\n        margin: 0 auto;\n    }\n\n    .form-row {\n        display: flex;\n        justify-content: space-between;\n        margin-bottom: 10px;\n    }\n\n    .form-row input {\n        width: 30%;\n        border-radius: 5px;\n        padding: 5px;\n    }\n\n    .form-row label {\n        color: #3498db;\n        font-weight: bold;\n    }\n\n    .form-textarea {\n        margin-bottom: 10px;\n    }\n\n    .form-submit {\n        display: block;\n        width: 30%;\n        margin: 0 auto;\n        padding: 10px;\n        border: none;\n        border-radius: 10px;\n        background-color: #3498db;\n        color: white;\n        font-weight: bold;\n        cursor: pointer;\n    }\n</style>\n\n<div class="form-container">\n    <form>\n        <div class="form-row">\n            <label for="first-name">First Name</label>\n           

In [132]:
display(HTML('```html\n<style>\n    .form-container {\n        background-color: #f2f2f2;\n        border-radius: 20px;\n        padding: 20px;\n        width: 50%;\n        margin: 0 auto;\n    }\n\n    .form-row {\n        display: flex;\n        justify-content: space-between;\n        margin-bottom: 10px;\n    }\n\n    .form-row input {\n        width: 30%;\n        border-radius: 5px;\n        padding: 5px;\n    }\n\n    .form-row label {\n        color: #3498db;\n        font-weight: bold;\n    }\n\n    .form-textarea {\n        margin-bottom: 10px;\n    }\n\n    .form-submit {\n        display: block;\n        width: 30%;\n        margin: 0 auto;\n        padding: 10px;\n        border: none;\n        border-radius: 10px;\n        background-color: #3498db;\n        color: white;\n        font-weight: bold;\n        cursor: pointer;\n    }\n</style>\n\n<div class="form-container">\n    <form>\n        <div class="form-row">\n            <label for="first-name">First Name</label>\n            <input type="text" id="first-name" name="first-name">\n        </div>\n        <div class="form-row">\n            <label for="last-name">Last Name</label>\n            <input type="text" id="last-name" name="last-name">\n        </div>\n        <div class="form-row">\n            <label for="email">Email Address</label>\n            <input type="email" id="email" name="email">\n        </div>\n        <div class="form-textarea">\n            <label for="message">Custom Message</label>\n            <textarea id="message" name="message" rows="4"></textarea>\n        </div>\n        <button type="submit" class="form-submit">Submit</button>\n    </form>\n</div>\n```'))

In [133]:
chat_llm_chain.predict(human_input="""the 3 text fields are NOT ON THE SAME ROW, please fix""")

'```html\n<style>\n    .form-container {\n        background-color: #f2f2f2;\n        border-radius: 20px;\n        padding: 20px;\n        width: 50%;\n        margin: 0 auto;\n    }\n\n    .form-row {\n        display: flex;\n        justify-content: space-between;\n        margin-bottom: 10px;\n    }\n\n    .form-row input {\n        width: 30%;\n        border-radius: 5px;\n        padding: 5px;\n    }\n\n    .form-row label {\n        color: #3498db;\n        font-weight: bold;\n    }\n\n    .form-textarea {\n        margin-bottom: 10px;\n    }\n\n    .form-submit {\n        display: block;\n        width: 30%;\n        margin: 0 auto;\n        padding: 10px;\n        border: none;\n        border-radius: 10px;\n        background-color: #3498db;\n        color: white;\n        font-weight: bold;\n        cursor: pointer;\n    }\n</style>\n\n<div class="form-container">\n    <form>\n        <div class="form-row">\n            <label for="first-name">First Name</label>\n           

In [134]:
display(HTML('```html\n<style>\n    .form-container {\n        background-color: #f2f2f2;\n        border-radius: 20px;\n        padding: 20px;\n        width: 50%;\n        margin: 0 auto;\n    }\n\n    .form-row {\n        display: flex;\n        justify-content: space-between;\n        margin-bottom: 10px;\n    }\n\n    .form-row input {\n        width: 30%;\n        border-radius: 5px;\n        padding: 5px;\n    }\n\n    .form-row label {\n        color: #3498db;\n        font-weight: bold;\n    }\n\n    .form-textarea {\n        margin-bottom: 10px;\n    }\n\n    .form-submit {\n        display: block;\n        width: 30%;\n        margin: 0 auto;\n        padding: 10px;\n        border: none;\n        border-radius: 10px;\n        background-color: #3498db;\n        color: white;\n        font-weight: bold;\n        cursor: pointer;\n    }\n</style>\n\n<div class="form-container">\n    <form>\n        <div class="form-row">\n            <label for="first-name">First Name</label>\n            <input type="text" id="first-name" name="first-name">\n            <label for="last-name">Last Name</label>\n            <input type="text" id="last-name" name="last-name">\n            <label for="email">Email Address</label>\n            <input type="email" id="email" name="email">\n        </div>\n        <div class="form-textarea">\n            <label for="message">Custom Message</label>\n            <textarea id="message" name="message" rows="4"></textarea>\n        </div>\n        <button type="submit" class="form-submit">Submit</button>\n    </form>\n</div>\n```'))